In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error, make_scorer
from sklearn.decomposition import PCA

In [6]:
path_archivo = "../input_data/combi_500_100.csv"
path_rand1 = "../input_data/uni_52500_1.csv"
path_rand2 = "../input_data/uni_52500_2.csv"
path_rand3 = "../input_data/uni_144489.csv"
path_rand4 = "../input_data/uni_1414886.csv"
path_rand5 = "../input_data/uni_2398582.csv"
datos_combi  = pd.read_csv(path_archivo)
datos_rand1 = pd.read_csv(path_rand1)
datos_rand2 = pd.read_csv(path_rand2)
datos_rand3 = pd.read_csv(path_rand3)
datos_rand4 = pd.read_csv(path_rand4)
datos_rand5 = pd.read_csv(path_rand5)

In [7]:
datos_combi = datos_combi.sort_values(by=["eta", "beta", "phi_K"]).reset_index(drop=True)

In [8]:
N_eig = 4
target = "phi_K"
def preprocesar_datos(d_frame, N_eig, target, opt = True):
    features_eig = list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
    features_dim = ["eta", "beta"]
    feature_especial = ["eig_0"]
    features_tot = [target] + feature_especial + features_dim + features_eig
    dat_copy = d_frame.copy()
    for i in range(N_eig):
        key_mod = "eig_" + str(i+1)
        prev_key = "eig_" + str(i)
        if i == 0:  
            dat_copy[key_mod] = 1/d_frame[key_mod] if opt else d_frame[prev_key]/d_frame[key_mod]
        else:
            dat_copy[key_mod] = d_frame[prev_key]/d_frame[key_mod]
        #fin if 
    #fin for
    for i in range(N_eig):
        col = "eig_" + str(i+1)
        #dat_copy[col] = np.log(dat_copy[col])
    #fin for 
    dat_copy = dat_copy.dropna()
    try:
        problematic_rows = dat_copy[(~np.isfinite(dat_copy)).any(axis=1) | (dat_copy.abs() > np.finfo(np.float64).max).any(axis=1)]
        problematic_indices = problematic_rows.index
        dat_copy = dat_copy.drop(index=problematic_indices)
    except:
        pass
    
    return dat_copy[features_tot]
#fin procesar_datos

In [9]:
datos_rand3

,phi_K,eta,beta,eig_0,eig_1,eig_2,eig_3,eig_4,eig_5,eig_6,...,eig_90,eig_91,eig_92,eig_93,eig_94,eig_95,eig_96,eig_97,eig_98,eig_99
0,0.400414,1.487623,0.317343,0.004936,2.046305,3.589234,6.437924,7.240966,18.031626,22.196340,...,8666.898871,8750.952238,8918.382676,9038.019736,9208.664469,9284.619575,9537.750184,9604.389903,9700.418903,10065.606823
1,1.475905,0.697588,1.948742,0.037546,2.578149,7.455840,7.797464,8.538153,13.416103,25.159900,...,1423.355458,1474.877175,1527.151366,1548.607590,1592.357815,1646.758081,1719.589292,1767.415069,1792.810724,1869.875301
2,1.327812,1.125665,2.372885,0.602727,1.516095,1.771436,3.015211,4.707727,5.394174,6.759736,...,110.726397,110.742632,113.338994,114.935644,115.729999,119.480752,121.249146,126.695055,126.790071,127.010828
3,0.621719,1.360050,2.402921,0.967858,1.339681,1.852271,3.292508,3.364588,3.789444,4.357686,...,51.756679,52.728158,52.741891,53.437610,54.842456,55.987554,57.236378,57.317893,57.726455,57.962549
4,0.771298,1.455346,0.571932,0.044228,1.793986,2.988463,5.574325,6.504536,12.605016,18.120249,...,1455.198311,1490.723942,1547.861095,1576.093974,1598.253491,1607.367691,1651.887260,1658.131471,1671.338133,1678.193950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50787,0.323387,0.759702,1.662409,0.043202,1.881281,3.265840,6.366387,8.124974,10.525084,13.220699,...,608.480760,622.837986,635.789151,651.802261,659.056243,668.507476,672.635765,689.556742,710.175600,711.079737
50788,1.546727,1.403701,2.766449,0.268222,1.176433,1.404041,2.591480,3.655583,4.743149,5.489248,...,243.965215,244.436072,252.717663,252.833569,254.003797,256.545260,258.651638,260.716998,270.839646,271.230526
50789,1.431851,0.452884,1.306711,0.003134,6.989902,8.299631,15.885702,33.696082,42.402321,42.884304,...,15564.215158,15685.350758,15864.716528,15907.078815,16112.467298,16173.100272,16837.425535,16953.059029,16966.556300,17610.212759
50790,1.411867,1.391845,1.952733,0.737192,1.598177,1.758049,2.382047,3.342095,3.819817,4.143118,...,93.862762,94.329584,94.382724,96.598343,101.210529,101.581543,101.759528,104.491600,105.164011,105.866568


In [10]:
datos_combi = preprocesar_datos(datos_combi, N_eig, target)
datos_rand1 = preprocesar_datos(datos_rand1, N_eig, target)
datos_rand2 = preprocesar_datos(datos_rand2, N_eig, target)
datos_rand3 = preprocesar_datos(datos_rand3, N_eig, target)
datos_rand4 = preprocesar_datos(datos_rand4, N_eig, target)
datos_rand5 = preprocesar_datos(datos_rand5, N_eig, target)

In [11]:
datos_rand3

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4
0,0.400414,0.004936,1.487623,0.317343,0.488686,0.570123,0.557514,0.889097
1,1.475905,0.037546,0.697588,1.948742,0.387875,0.345789,0.956188,0.913249
2,1.327812,0.602727,1.125665,2.372885,0.659589,0.855856,0.587500,0.640481
3,0.621719,0.967858,1.360050,2.402921,0.746446,0.723264,0.562571,0.978577
4,0.771298,0.044228,1.455346,0.571932,0.557418,0.600304,0.536112,0.856990
...,...,...,...,...,...,...,...,...
50787,0.323387,0.043202,0.759702,1.662409,0.531553,0.576048,0.512982,0.783558
50788,1.546727,0.268222,1.403701,2.766449,0.850027,0.837891,0.541791,0.708910
50789,1.431851,0.003134,0.452884,1.306711,0.143064,0.842194,0.522459,0.471441
50790,1.411867,0.737192,1.391845,1.952733,0.625713,0.909063,0.738041,0.712741


In [12]:
metadata_temporal = {"combi": len(datos_combi), "rand1": len(datos_rand1), "rand2": len(datos_rand2)}
print(metadata_temporal)

{'combi': 52500, 'rand1': 52494, 'rand2': 52494}


In [13]:
N_phi_K = 500
N_datos = len(datos_combi)
N_partes = int(N_datos/N_phi_K)
print(N_partes)

105


In [14]:
def graficar_lambdas(data_frame, target, n_eig, preds = {}, op = lambda x: x, nombre = "lineal/", nombre_feat = "lambda_moño"):
    cols_eig = list(map(lambda x: "pca" + str(x), range(n_eig)))
    cols_preds = list(map(lambda x: "pred_" + x, cols_eig)) if len(preds) != 0 else []
    eta = str(np.array(data_frame["eta"])[0]/np.pi)[:4] + "pi"
    beta = str(np.array(data_frame["beta"])[0]/np.pi)[:4] + "pi"
    x = data_frame[target]
    ys = op(data_frame[cols_eig])
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(x, ys)
    if len(preds) != 0:
        ax.plot(preds["y"], ys)
        nombre = "pred/" + nombre
    #fin if 
    ax.set_xlabel(target)
    ax.set_ylabel(nombre_feat)
    ax.set_title("eta=" +  eta + ", beta=" + beta)
    ax.legend(cols_eig + cols_preds)
    plt.savefig("pictures/" + nombre + "eta=" +  eta + ",beta=" + beta + "_" + ".png")
    plt.close()
#fin for 

In [15]:
def graficar_todas(d_frame, n_eig, target, op = lambda x: x, preds = {}, nombre = "lineal/", nombre_feat = "l_moño"):
    for n in range(N_partes):
        start = N_phi_K*n
        end = N_phi_K*(n+1)
        ys = preds["y"][start:end] if len(preds) != 0 else []
        preds_fin = {"y": ys, "X": preds["X"].iloc[start:end]} if len(preds) != 0 else {}
        graficar_lambdas(d_frame.iloc[start:end], target, n_eig, preds_fin, op, nombre, nombre_feat)
    #fin for 
#fin función

In [16]:
def get_metrics(X, y, model):
    y_gorro = model.predict(X)
    R2 = r2_score(y, y_gorro)
    RMSE = root_mean_squared_error(y, y_gorro)
    MAE = mean_absolute_error(y, y_gorro)
    return {"R2": R2, "RMSE": RMSE, "MAE": MAE}
#fin función

class TransformadorDeDatos(BaseEstimator, TransformerMixin):
    def __init__(self, cols_tr, deg = 3):
        self.deg = deg
        self.cols_tr = cols_tr
    #fin init

    def fit(self, X, y=None):
        return self
    #fin fit

    def transform(self, X):
        datos = X.copy()
        cols_transformar = self.cols_tr
        poly = PolynomialFeatures(degree=self.deg, include_bias=False)
        poly_data = poly.fit_transform(datos[cols_transformar])
        poly_feature_names = poly.get_feature_names_out(input_features=cols_transformar)
        datos_finales = pd.DataFrame(poly_data, columns=poly_feature_names)
        return datos_finales
    #fin transform
#fin class

def crear_pipeline(modelo, grado, nombre_modelo, cols_tr):
    pipeline = Pipeline([('custom-transformer', TransformadorDeDatos(cols_tr, deg = grado)),
            #('power-transformer', PowerTransformer(method = "box-cox", standardize = True)), 
            (nombre_modelo, modelo)])
    return pipeline
#fin función

In [17]:
orden_pol = 3
features = ["eta", "beta"] + list(map(lambda x: "eig_" + str(x+1), range(N_eig)))
pipeline = crear_pipeline(LinearRegression(), orden_pol, "re_lineal_simple", features)

In [18]:
X_combi = datos_combi[features]
X_rand1 = datos_rand1[features]
X_rand2 = datos_rand2[features]
X_rand3 = datos_rand3[features]
X_rand4 = datos_rand4[features]
X_rand5 = datos_rand5[features]
y_combi = datos_combi[target]
y_rand1 = datos_rand1[target]
y_rand2 = datos_rand2[target]
y_rand3 = datos_rand3[target]
y_rand4 = datos_rand4[target]
y_rand5 = datos_rand5[target]

In [19]:
X_train = pd.concat((X_combi, X_rand2, X_rand3, X_rand4, X_rand5), axis = 0)
y_train = pd.concat((y_combi, y_rand2, y_rand3, y_rand4, y_rand5), axis = 0)

In [20]:
pipeline.fit(X_train, y_train)

Pipeline(steps=[('custom-transformer',
                 TransformadorDeDatos(cols_tr=['eta', 'beta', 'eig_1', 'eig_2',
                                               'eig_3', 'eig_4'])),
                ('re_lineal_simple', LinearRegression())])

In [21]:
#X_test1 = pd.concat((X_rand1, X_rand5), axis = 0)
#y_test1 = pd.concat((y_rand1, y_rand5), axis = 0)
metrs_train = get_metrics(X_train, y_train, pipeline)
metrs_test = get_metrics(X_rand1, y_rand1, pipeline)

In [22]:
print(metrs_train)
print(metrs_test)

{'R2': 0.6489747568623838, 'RMSE': 0.26849766656131974, 'MAE': 0.20840350617542924}
{'R2': 0.6477060561309156, 'RMSE': 0.26923092908317753, 'MAE': 0.2094645445373657}


In [23]:
path_archivo_kg = "../input_data/KG_random.csv"
str_to_ndarray = lambda x: np.fromstring(x, sep=' ')
datos_kg_random = pd.read_csv(path_archivo_kg, converters={'eigvals': str_to_ndarray})

In [24]:
datos_kg_random = datos_kg_random[datos_kg_random["shape"] == "parallelepiped"].copy()

In [25]:
del datos_kg_random["rho"]

In [26]:
datos_kg_random

,K,G,dx,dy,dz,shape,eigvals
98304,1.137564,0.925310,0.345791,0.722980,0.454774,parallelepiped,"[-1.5547410733937762e-13, -6.69847392884328e-1..."
98305,5.500412,5.321621,0.402060,0.999638,0.807374,parallelepiped,"[-2.1667717054444154e-12, -2.0543006395375965e..."
98306,3.375617,3.980662,0.429638,0.851665,0.529495,parallelepiped,"[-4.817232318550446e-13, -3.323750112763421e-1..."
98307,1.448420,4.074601,0.151318,0.942633,0.328977,parallelepiped,"[-2.6737693556924537e-12, -2.351597437975512e-..."
98308,0.351677,5.407791,0.446276,0.792512,0.989446,parallelepiped,"[-1.6759891759031827e-13, -3.9777497738663696e..."
...,...,...,...,...,...,...,...
131067,0.603325,3.002608,0.720661,0.439132,0.152189,parallelepiped,"[-2.9206283664231715e-12, -7.439679690463981e-..."
131068,2.302202,2.474801,0.132153,0.607512,0.605882,parallelepiped,"[-3.9535462097488675e-13, -1.163763376545102e-..."
131069,4.789391,3.091901,0.174914,0.402842,0.230977,parallelepiped,"[-5.3840655925037205e-12, -2.069314763230146e-..."
131070,0.920026,0.420177,0.185685,0.716565,0.322597,parallelepiped,"[-4.557497223449767e-13, -3.4356261396269377e-..."


In [27]:
for q in range(N_eig+1):#nondeg_minlen):
    datos_kg_random['eig_' + str(q)] = datos_kg_random['eigvals'].apply(lambda arr: arr[6 + q]) 

datos_kg_random = datos_kg_random.drop(columns=['eigvals'])

In [28]:
datos_kg_random.head()

,K,G,dx,dy,dz,shape,eig_0,eig_1,eig_2,eig_3,eig_4
98304,1.137564,0.925310,0.345791,0.722980,0.454774,parallelepiped,3.344897,4.253522,5.383594,10.070057,10.958655
98305,5.500412,5.321621,0.402060,0.999638,0.807374,parallelepiped,6.564257,10.320303,18.541555,20.077801,24.790483
98306,3.375617,3.980662,0.429638,0.851665,0.529495,parallelepiped,10.756621,12.849225,15.328886,29.027839,33.703187
98307,1.448420,4.074601,0.151318,0.942633,0.328977,parallelepiped,1.659110,5.366767,5.584904,10.405840,17.398203
98308,0.351677,5.407791,0.446276,0.792512,0.989446,parallelepiped,2.555409,4.226163,4.688969,7.011962,7.698858


In [29]:
datos_kg_random["phi_K"] = np.arctan(datos_kg_random["G"]/datos_kg_random["K"])
indexes_kg_random = [98304, 131071]
datos_kg_random["eta"] = np.ones(len(datos_kg_random))
datos_kg_random["beta"] = np.ones(len(datos_kg_random))
for i in range(indexes_kg_random[0], indexes_kg_random[1] + 1):
    lis_or = np.sort([datos_kg_random["dx"][i], datos_kg_random["dy"][i], datos_kg_random["dz"][i]])
    r = (lis_or[0]**2 + lis_or[1]**2 + lis_or[2]**2)**0.5
    datos_kg_random.loc[i, "eta"] = 2*np.arccos(lis_or[2]/r)
    datos_kg_random.loc[i, "beta"] = 4*np.arctan(lis_or[0]/lis_or[1])
#fin for 
datos_kg_random = datos_kg_random.copy()

In [30]:
datos_kg_random_copy = preprocesar_datos(datos_kg_random, N_eig, target, opt = False)

In [31]:
datos_kg_random_copy

,phi_K,eig_0,eta,beta,eig_1,eig_2,eig_3,eig_4
98304,0.682866,3.344897,1.337486,2.600392,0.786383,0.790090,0.534614,0.918914
98305,0.768879,6.564257,1.468138,1.848136,0.636053,0.556604,0.923485,0.809900
98306,0.867462,10.756621,1.350259,2.726644,0.837142,0.838236,0.528075,0.861279
98307,1.229252,1.659110,0.733530,1.724437,0.309145,0.960942,0.536709,0.598099
98308,1.505856,2.555409,1.486674,2.051426,0.604664,0.901299,0.668710,0.910779
...,...,...,...,...,...,...,...,...
131067,1.372503,2.463035,1.145569,1.334456,0.531536,0.528263,0.606136,0.973601
131068,0.821514,2.507252,1.591347,0.859013,0.474287,0.873008,0.471576,0.996856
131069,0.573257,36.155819,1.247024,2.592575,0.836996,0.772776,0.496226,0.869119
131070,0.428413,1.034104,0.958174,2.089119,0.812896,0.630602,0.401596,0.997046


In [32]:
X_test3 = datos_kg_random_copy[features]
y_test3 = datos_kg_random_copy[target]

In [33]:
metrs3 = get_metrics(X_test3, y_test3, pipeline)

In [34]:
print(metrs3)

{'R2': -0.37553270194899824, 'RMSE': 0.4306567763844785, 'MAE': 0.26098580472744626}
